<a href="https://colab.research.google.com/github/redinbluesky/nlp-with-transformers/blob/main/03-트랜스포머 파헤치기.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  텍스트 분류 목차
* [Chapter 0 개요](#chapter0)
* [Chapter 1 트랜스포머 마키텍처](#chapter1)
* [Chapter 2 인코더](#chapter2)
    * [Chapter 2-1 셀프 어텐션](#chapter2-1)

## Chapter 0 개요 <a class="anchor" id="chapter0"></a>
1. 이 장에서는 트랜스포머 모델의 주요 구성 요소와 이를 파이토치로 구현하는 방법을 살펴본다.
   - 텐서플로우로 동일한 작업을 수행하는 방법도 알아본다.

2. 어텐션 메커니즘을 만드는데 초점을 맞춘 다음, 트랜스포머 인코더를 구현하기 위한 필요 요소를 추가한다.

3. 문제를 해결하기 위해서 트랜스포머를 사용하고 모델을 미세 튜닝하는데 트랜스포머 아키텍처의 기술적인 측면을 이해할 필요는 없다.
   - 하지만 기술적인 측면을 알면 트랜스포먼의 한계점을 파악하거나 새로운 도메일에 적용할 때 도움이 된다.


## Chapter 1 트랜스포머 아키텍처 <a class="anchor" id="chapter1"></a>
1. 트랜스포머는 인코더-디코더 구조를 기반으로 한다.
    - 인코더는 입력 시퀀스를 처리하고 디코더는 출력 시퀀스를 생성한다.
    - 시퀸스를 다른 언어로 번역하는 기계 번역과 같은 작업에 주로 사용된다.
2. 인코더
    - 입력 토큰의 시퀸스를 은닉 상태또는 문맥이라 부르는 임베딩 벡터의 시퀸스로 변환한다.

3. 디코더
   - 인코더의 은닉 상태를 사용해 출력 토큰의 시퀸스를 한 번에 하나씩 반복적으로 생성한다.

4. 이미지로 표현하면 아래와 같다.
    - 입력 텍스트를 토큰화하고 토큰 임베딩으로 변환한다.
    - 어텐션 매커니즘은 토큰의 상대적 위치를 알지 못한다 따라서 텍스트의 순서 특징을 모델일하기 위해 입력에 토큰 위치 임베딩을 더한다.
    - 인코더가 인코더 층의 스택 또는 '블록'으로 구성되고, 디코더도 마찬가지로 디코더 층의 스택으로 구성된다.
    - 디코더 층마다 인코더의 출력이 주입된다. 디코더는 시퀸스에서 가장 가능성이 있는 다음 토큰을 예측한다.
    - 이전 단계의 출력이 다음 단계의 입력으로 사용되며 EOS와 같은 특수 토큰이 생성되면 프로세스가 중지된다.

        ![transformer](image/03_01_transformer.png)

5. 인코더와 디코더 블록은 독립적인 모델이 되었다.
    - 인코더 유형
        - 텍스트 시퀸스의 입력을 풍부한 수치 표현으로 변환한다.
        - 텍스트 분류나 개체명 인식 같은 작업에 잘 맞는다.
        - BERT, RoBERTa, DistiBERT 같은 모델이 인코더 유형이다.
        - 한 토큰에 대한 계산한 표현은 왼쪽(이전 토큰)과 오른쪽(이후 토큰)문맥에 따라 달라진다.
            - 이를 양방향 어텐션이라고 한다.
    - 디코더 유형
        - "Thanks for lunch, I had a ..."같은 시작 텍스트가 주어지면 "great time!"같은 후속 텍스트를 생성한다.
        - GPT, GPT-2, GPT-3 같은 모델이 디코더 유형이다
        - 한 토큰에 대한 계산한 표현은 오직 왼쪽 문캑에 따라 달라진다. 
            - 이를 코잘 어텐션 또는 자기회귀 어텐션이라고 한다.
    - 인코더-디코더 유형
        -  한 텍스트의 시퀸스를 다른 시퀸스로 매핑하는 복잡한 모델일에 사용한다.
        - 기계 번역, 요약, 질문 답변 같은 작업에 잘 맞는다.
        - T5, BART 같은 모델이 인코더-디코더 유형이다

## Chapter 2 인코더 <a class="anchor" id="chapter2"></a>
1. 인코더는 여러 개의 인코더 층이 서로 쌓여 구성된다.
    - 각 인코더 층은 임베딩 시퀸스를 받아 멀티 헤드 셀프 어텐션 층과 피드 포워드 신경망 층을 통과시킨다.
    - 인코더 응의 출력 임베딩은 입력과 크기가 동일하다.
    - 인코더 스텍의 주효 역확은 입력 임베딩을 업데이트해 시퀸스의 문맥 정보가 인코딩된 표현을 만드는 것이다.
    -  "apple"이라는 단어에 "keynote"나 "phone"이 가까이 있다면 "apple"에 대한 표현을 회사와는 가깝고 과일과는 멀게 업데이트 할 것이다.
    - 멀티 헤드 어텐션 층과 피드 포워드 층은 심층 신경암을 효율적으로 훈련하기 위해 스킵 연결과 층 정규화를 사용한다.

        ![transformer-encoder](image/03_02_transformer_encoder.png)

### Chapter 2-1 셀프 어텐션 <a class="anchor" id="chapter2-1"></a>
1. 어텐션 메커님즘을 사용하여 신경망이 시퀸스의 각 원소에 다른 양의 가중치 또는 어텐션을 할당한다.
    - 텍스트 시퀸스는 토큰임베딩이며, 각 토큰은 고장차원의 벡터에 매핑된다.
    - 셀프 어텐션의 셀프는 이 가중치가 동일 집합에 있는 모든 은닉 상태에 대해 계산된다는 사실을 나타낸다.
    - 순환 모델과 연관된 어텐션 메커니즘은 특정 디코딩 타임스텝에서 해당 디코더의 은닉 상태와 인코더의 각 은닉 상태가 가진 관련성을 계산한다.

2. 각 토큰에 대한 고정된 임베딩을 사용하는 대신 전체 시퀸스를 사용해 각 임베딩의 가중 경균을 계산하는 것이 셀프 어텐션의 기본 개념이다.
    - 토큰 임베딩의 시퀸스 x = (x₁, x₂, ..., xₙ)가 주어지면 셀프 어텐션은 새로운 임베딩 시퀸스 x' = (x'₁, x'₂, ..., x'ₙ)를 생성한다.
    - 여기서 각 x'ᵢ는 x의 선형 결합이다.
        - x'ᵢ = Σⱼ wᵢⱼ xⱼ
        - wᵢⱼ는 어텐션 가중치라고 하며 Σⱼ wᵢⱼ = 1이 되도록 정규화 된다.

3. 토큰 임베딩의 평균을 구하는 이유는 다음과 같다.
    - "flies"를 보면 파리를 뜻하는 명사 "fly"가 떠오를 수 있다. 
    - 하지만 "time flies like an arrow"과 같은 문맥이 주어지면 날아간다는 뜻의 동사 "flies"를 떠올린다.
        - 'time"과 'arrow"는 "flies"와 더 관련이 있기 때문에 더 높은 가중치를 받는다.
    - 이처럼 모든 토큰 임베딩을 비율을 달리해 통합하면 문맥을 내포하는 "flies"표현이 만들어진다.
    - 이런식으로 생성된 임베딩을 문맥 고려 임베딩이라고 한다.
    - 아래의 그림에서 문맥에 따라 셀프 어텐션을 통해 "flies"의 다른 표현 이 생성되는 것을 볼 수 있다.

        ![self-attention](image/03_03_self_attention.png)
